### Let's try HP Tuning on this too, which can help us later

In [ ]:
%%capture
!pip install smdebug torch torchvision tqdm

In [ ]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts
from smdebug import modes
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import IPython

In [ ]:
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([16, 32, 64, 128, 256]),
}

role = sagemaker.get_execution_role()

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [ ]:
estimator = PyTorch(
    entry_point="train_model.py",
    base_job_name='test_bins',
    role=role,
    framework_version="1.4.0",
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    py_version='py3'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=2,
    objective_type=objective_type
)

In [ ]:
os.environ['SM_CHANNEL_TRAINING']='s3://sagemaker-studio-0qajs6acpsz/BinDataset/'
os.environ['SM_MODEL_DIR']='s3://sagemaker-studio-0qajs6acpsz/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://sagemaker-studio-0qajs6acpsz/output/'
tuner.fit({"training": "s3://sagemaker-studio-0qajs6acpsz/BinDataset/"})